In [1]:
## Importing Necessary Modules 

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
## getting the appropriate website parsed into hmtl
### Defining a function which takes in a url and outputs soup
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return(soup)


In [75]:
def extract_result(url):
    
    #soupify
    soup = get_soup(url)
    
    headers = []
    data = []
    
    #find rows
    rows = soup.find_all('tr')
    
    #generate header
    header_row = rows[0]
    for header in header_row.find_all('th'):
        headers.append(header) 
        data.append([])
    
    #generate rows
    for data_row in rows[1:]:
        row = data_row.find_all('td')
        for i in range(len(row)):
            if len(row) != len(data):
                continue
            else:
                data[i].append(row[i].text.strip())
    
    #put into dataframe
    data_dict = {}
    for i in range(len(headers)):
        data_dict[headers[i]] = data[i]
    df = pd.DataFrame(data_dict)
    return(df)


In [77]:
all_stages_data = {}
stage_number = 1
url_basic = 'https://www.procyclingstats.com/race/tour-de-france/2023/stage-'
for stage_number in range(1,22):
    url = url_basic + str(stage_number)
    all_stages_data[str(stage_number)] = extract_result(url)

    
print(all_stages_data['20'])

    [Rnk] [GC] [Timelag] [BIB] [H2H] [Specialty]  \
0       1    2     +7:29    11           Climber   
1       2    8    +16:09    95           Climber   
2       3    1     +0:00     1                GC   
3       4    4    +12:23   161           Climber   
4       5    3    +10:56    19           Climber   
..    ...  ...       ...   ...   ...         ...   
296   146  146         -   144                TT   
297   147  148        ▲1   185           Classic   
298   148  149        ▲1   194           Classic   
299   149  150        ▲1   192            Sprint   
300   150  151        ▲1    58            Sprint   

                                    [Rider] [Age]                 [Team]  \
0           POGAČAR Tadej UAE Team Emirates    24      UAE Team Emirates   
1              GALL Felix AG2R Citroën Team    25      AG2R Citroën Team   
2              VINGEGAARD Jonas Jumbo-Visma    26            Jumbo-Visma   
3              YATES Simon Team Jayco AlUla    30       Team Jayco AlUl

In [73]:
def get_race_info(url):
    
    soup = get_soup(url)
    
    ul_elements = soup.find('ul', class_='infolist')
    ul_points = ul_elements.find_all('li')
    
    results = {}
    
    for point in ul_points:
        divs = point.find_all('div')
        
        key = divs[0].text.strip().replace(':','')
        val = divs[1].text
        
        results[key] = val

    return results

print(get_race_info('https://www.procyclingstats.com/race/tour-de-france/2023/stage-20'))

{'Date': '22 July 2023', 'Start time': '13:45 ', 'Avg. speed winner': '38.64 km/h', 'Race category': 'ME - Men Elite', 'Distance': '133.5 km', 'Points scale': 'GT.A.Stage', 'UCI scale': 'UCI.WR.GT.A.Stage', 'Parcours type': '', 'ProfileScore': '303', 'Vertical meters': '3471', 'Departure': 'Belfort', 'Arrival': 'Le Markstein', 'Race ranking': '1', 'Startlist quality score': '1584', 'Won how': 'Sprint of 5 riders', 'Avg. temperature': '22 °C'}
